The argparse module is part of the Python standard library, and lets your code accept command line arguments. This makes your code easy to configure at run-time. There are multiple ways to do this in Python, but argparse is the most powerful with minimal additional code required. This article is a collection of example code snippets — I wrote it because the official documentation is quite heavy-going, and couldn’t find a suitable quick reference.

Basic usage
There are four basic steps to using argparse in your code:

* import the argparse module;
* create the parser;
* add arguments;
* parse the arguments.

In [ ]:
import argparse

# Create the parser and add arguments
parser = argparse.ArgumentParser()
parser.add_argument(dest='argument1', help="This is the first argument")

# Parse and print the results
args = parser.parse_args()
print(args.argument1)

This adds a single positional, mandatory (required) argument. If we run our script but forget to add the argument, we will get an error. If we supply an argument, it gets printed to the console:

```zsh
$ python argparse_basic.py
usage: argparse_basic.py [-h] argument1
argparse_basic.py: error: the following arguments are required: argument1
$ python argparse_basic.py "Hello, world!"
Hello, world!
$ python argparse_basic.py 3.14159
3.14159
```

We can also run the script and print the help message, which will list the arguments and any constraints:

```zsh
$ python basic.py --help
usage: basic.py [-h] argument1
positional arguments:
  argument1   This is the first argument
optional arguments:
  -h, --help  show this help message and exit
```
The following sections expand on the code snippet above — they all use a similar structure, although we don’t reproduce all of the code unless it adds value.


## Positional arguments 

These are read from the command line in the order that they appear after the script name, and are required unless otherwise specified:

In [ ]:
parser.add_argument(dest='argument1', type=str, help="A string argument")
parser.add_argument(dest='argument2', type=int, help="An integer argument")
parser.add_argument(dest='argument3', type=float, help="A float argument")

# Validate that the input is in specified list
parser.add_argument(dest='argument4', choices=['red', 'green', 'blue'])

# Accept multiple inputs for an argument, returned as a list
# Will be of type string, unless specified
parser.add_argument(dest='argument5', nargs=2, type=int)

# Optional positional argument (length 0 or 1)
parser.add_argument(dest='argument6', nargs='?')

## Optional (flag) arguments

Flag arguments are specified on the command line by name, using single and/or double dashes (e.g. -h and/or --help) and are optional unless otherwise specified. They can be added on the command line in any order:

In [ ]:
# Boolean flag (does not accept input data), with default value
parser.add_argument('-a1', action="store_true", default=False)

# Cast input to integer, with a default value
parser.add_argument('-a2', type=int, default=0)

# Provide long form name as well (maps to 'argument3' not 'a3')
parser.add_argument('-a3', '--argument3', type=str)

# Make argument mandatory
parser.add_argument('-a4', required=True)

# Retur the input via different parameter name
parser.add_argument('-a5', '--argument5', dest='my_argument')

args = parser.parse_args()
print(args.a1)
print(args.a2)
print(args.argument3)
print(args.a4)
print(args.my_argument)

## Mutually exclusive arguments

Some arguments don’t make sense to be both set at the same time. The mutually exclusive group allows you to specify this: an exception is raised if more than one argument in the group is provided:

In [ ]:
group = parser.add_mutually_exclusive_group()
group.add_argument('--arg1', action='store_true')
group.add_argument('--arg2', action='store_false')

according to above example:

1. `arg1` is `True` if the flag is included on the command line, otherwise `False`. 
2. `arg2` is the opposite.

## Advanced functionality
The examples above should cover more than 90% of the scenarios that you are are likely to use argparse for. It has some rather powerful additional features, however, and in the following sections we’ll see how it can be used to perform more specific input validation as well as managing input and output streams.

### Parsing custom data
As well as using the predefined types such as str, int, float etc, the type= parameter can accept any callable that takes a single string argument and returns a value. This can be used to do custom validation or conversion of the input:

In [ ]:
def single_word(string):
    # Check input does not contain spaces
    if (' ' in string):
        msg = f'\"{string}\" is not a single word'
        raise argparse.ArgumentTypeError(msg)
    return string

parser.add_argument('argument1', type=single_word)

In the example above, an exception is raised if the input contains one or more space characters:

```shell
$ python argparse_custom_type.py abc123
abc123

$ python argparse_custom_type.py "Hello, world!"
usage: argparse_custom_type.py [-h] argument1
error: argument1: "Hello, world!" is not a single word
```

### Opening and closing files

It is relatively common to use command line arguments to specify paths to input and output files, such as for source data and results summaries. `argparse` can handle that for you: ***FileType*** gives you a more flexible way of specifying that an argument should be a file, and can handle encoding, access mode (read/write/append etc) and other hyperparameters:

In [ ]:
import argparse

parser = argparse.ArgumentParser()

# Add a required, positional argument for the input data file name,
# and open in 'read' mode
parser.add_argument('infile', type=argparse.FileType('r'))

# Add an optional argument for the output file,
# open in 'write' mode and and specify encoding
parser.add_argument('--output', type=argparse.FileType('w', encoding='UTF-8'))

args = parser.parse_args()

# Read a CSV file,  sum the values in the second column,
# and optionally write to file
sum = 0
with args.infile as infile:
    for line in infile:
        value = int(line.split(',')[1])
        sum += value
        print(sum)
    
        if args.output is not None:
            args.output.writelines(f'{sum}\n')

When using the FileType type, argparse takes care of opening the file(s) for you. If the input file name is not found, it throws an error that references the associated argument; if the optional output file argument is supplied, argparse creates the output stream:

```bash
$ python argparse_FileType.py source_dta.csv
usage: argparse_FileType.py [-h] [--output OUTPUT] infile
argparse_FileType.py.py: error: argument infile: can't open 'source_dta.csv': [Errno 2] No such file or directory: 'source_dta.csv'
$ python basic.py source_dta.csv --output sum.csv
```